In [280]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from urllib.request import urlopen
from bs4 import BeautifulSoup

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [223]:
!pip install folium

### Defining a function to get HTML contect from the webpage

In [261]:
def getHTMLContent(link):
    html = urlopen(link)
    soup = BeautifulSoup(html, 'html.parser')
    return soup

### Extracting relevant table contents from the html file

In [262]:
content = getHTMLContent('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
tables = content.find_all('table')

In [263]:
table = content.find('table', {'class': 'wikitable sortable'})

### creating a list of all table rows

In [264]:
data = []
rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

In [265]:
del data[0]

### Removing rows acontaining "Not assigned" values in Borough

In [266]:
newdata=[]
for i in range(0,len(data)):
    if data[i][1]!="Not assigned":
      newdata.append(data[i])
        

### Allocating value to "Not assigned" values in Neighborhood 

In [267]:
for i in range(0,len(newdata)):
    if newdata[i][2]=="Not assigned":
        newdata[i][2]=newdata[i][1]

### Joining Neighborhood values with ',' for same PostalCode

In [268]:
f_newdata=[]
n=len(newdata)
i=0
while i<n:
    p=newdata[i][0]
    #s=newdata[i][2]
    j=i+1
    while j<n and newdata[j][0]==p :
        newdata[i][2]=newdata[i][2]+","+newdata[j][2]
        j=j+1
    f_newdata.append(newdata[i])
    i=j

### Converting list to Dataframe

In [269]:
df=pd.DataFrame(f_newdata, columns=('PostalCode', 'Borough', 'Neighbourhood'))

In [270]:
df.sort_values('PostalCode', inplace=True)

### loading location data from csv file

In [271]:
location_data = pd.read_csv('http://cocl.us/Geospatial_data')

### Adding latitude and Longitude data to the dataframe

In [272]:
df.insert(3,"Latitude",location_data['Latitude'], True)
df.insert(4,"Longitude",location_data['Longitude'], True)

In [273]:
df.sort_index(inplace=True)

## Selecting boroughs which contain the word Toronto

In [274]:
n=df.shape[0]
df1 = pd.DataFrame(columns=df.columns)
i=0
while i<n:
    str=df.iloc[i,1]
    if str.find('Toronto')>-1:
        df1=df1.append(df.iloc[i],ignore_index=True)
        i=i+1
    i=i+1

In [275]:
df1

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.763573,-79.188711
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.692657,-79.264848
2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
3,M4E,East Toronto,The Beaches,43.786947,-79.385975
4,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259
5,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.737473,-79.464763
6,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.695344,-79.318389
7,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
8,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",43.659526,-79.340923
9,M4L,East Toronto,"The Beaches West,India Bazaar",43.704324,-79.388790


### Getting geographical coordinates of Toronto

In [281]:
from geopy.geocoders import Nominatim
import folium

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


### Creating a map of Toronto with neighbourhood locations

In [282]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_toronto

### Cluster Analysis

#### Defining Foursquare Credentials and Version

In [248]:
CLIENT_ID = 'A54AYVKDDOYWVQTJMVEEB3NI10XTR3VZU0RJQGF5QL1ZYB1C' # your Foursquare ID
CLIENT_SECRET = 'IIINL2WE1LYT0RX2SC4455CASV0HFS5V5MLVIN0OX43WDTPH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Creating a funtion to get top 100 vaenues in all neighbourhood

In [254]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT=100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [255]:
toronto_venues = getNearbyVenues(names=df1['Neighbourhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )

Harbourfront,Regent Park


IndexError: tuple index out of range

#### get the count of venues for each neighbourhood

In [257]:
toronto_venues.groupby('Neighbourhood').count()

NameError: name 'toronto_venues' is not defined

### Analyse each neighbourhood

In [ ]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

#### Grouping rows by neighbourhood

In [ ]:
manhattan_grouped = manhattan_onehot.groupby('Neighbourhood').mean().reset_index()

### Define a function to sort venues in descending order

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

###  clusters Neighbourhoods

In [ ]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

### Visualizating Clusters

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters